# Ch9-3. 산포 통계량

산포란 데이터가 얼마나 퍼져있는지를 의미한다.  
같은 확률 밀도 함수라고 했을 경우, 산포가 작은 변수는 한 값에 몰려있고, 산포가 큰 변수는 넓게 퍼져있다. 산포 통계량이란 데이터의 산포를 나타내는 통계량이다.  

## 분산과 표준 편차
- 편차 : 한 샘플이 평균으로부터 떨어진 거리 $(x_i - \mu)$  
- 분산 : 편차의 제곱의 평균  
$$ \frac {\Sigma^{n}_{(i=1)} (x_i - \mu)^2} {n-1} $$
이산 확률 변수에서 $$ {\displaystyle \operatorname {Var} (X)=\sum _{i=1}^{n}p_{i}\cdot (x_{i}-\mu )^{2}} $$
완전 연속 확률 변수에서 $${\displaystyle \operatorname {Var} (X)=\int _{-\infty }^{+\infty }x^{2}f(x)\,dx-\mu ^{2}}$$  
    - *참고 : https://ko.wikipedia.org/wiki/%EB%B6%84%EC%82%B0*
    - 편차의 합은 항상 0이 되기 때문에, 0이 되는 것을 방지하기 위해 제곱을 사용.  
    - 자유도가 0(모분산)이면 n-1로 나누지 않고 n으로 나눔.  
        - 불편통계량이라는 부분이라 어려워서 skip
- 표준편차 : 분산의 제곱근
    - 분산에서 제곱의 영향을 없앤 지표
    - 특별한 경우를 제외하고는 분산보다 많이 씀.
    
## 변동계수(coefficient of variation, CV)
분산과 표준편차 모두 값의 스케일에 크게 영향을 받아서, 상대적인 산포를 보여주는데 부적합하다. 
- 따라서 피처간의 비교, 상대적인 것을 보기 위해서는 변수를 스케일링한 뒤, 분산 혹은 표준편차를 구해야 한다.  
- 만약 모든 데이터가 양수인 경우에는 변돈계수(상대 표준편차)를 사용할 수 있다.  
    - 변동계수 : $ 표준편차 / 산술 평균 $  
        - 평균이 0인 경우 사용할 수 없음  
        - 만약 음수가 있다면 평균이 음수에 의해 작아지기 때문에 실제 산포와 거리가 멀어질 수 있어서, 모든 데이터가 양수인 경우가 아니면 잘 쓰이지 않는다. 



# 파이썬을 이용한 분산, 표준편차, 변동계수 계산

`ddof` 는 자유도를 의미한다. 아주아주 쉽게 생각하면, `ddof`가 1이면 $n-1$로 나누고 0이면 $n$으로 나누는 계산으로 생각해두면 된다. (라고 하셨지만... I want more) 특별한 경우를 제외하고 자유도는 1로 둔다. 


In [2]:
import numpy as np
from scipy.stats import *
import pandas as pd

## 분산 계산
```
numpy.var(x, ddof)
numpy.array(x).var(ddof)
Series(x).var(ddof)
```

In [3]:
x = [1, 2, 3, 4, 5]
print(np.var(x, ddof = 1))        # 분모 = n-1 (가장 일반적임)
print(np.array(x).var())          # 분모 = n (자유도를 설정하지 않음)
print(pd.Series(x).var(ddof = 0)) # 분모 = n

2.5
2.0
2.0


## 표준편차 계산

```
numpy.std(x, ddof)
numpy.array(x, ddof).std()
Series.std(x, ddof)
```

In [4]:
x = [1, 2, 3, 4, 5]
print(np.std(x, ddof = 1))        # 자유도 = 1
print(np.array(x).std())          # 자유도 = 0
print(pd.Series(x).std(ddof = 1)) # 자유도 = 1

1.5811388300841898
1.4142135623730951
1.5811388300841898


## 변동계수

```
numpy.std(x, ddof) / numpy.mean(x)
scipy.stats.variation(x)
```

### 변동계수의 필요성

스케일차이가 존재하면 그에 따라 표준편차가 커질 수 있어서, 상대적인 편차를 보기위해 변동계수가 필요하며, 변동계수를 사용하기 위해서는 모든 데이터 값들이 양수여야 한다.  

양수가 아닌데 보기 위해서는 따로 스케일링을 하고 보아야 한다고.

In [5]:
# 표준편차 비교

x1 = np.array([1, 2, 3, 4, 5])
x2 = x1 * 10

print(np.std(x1, ddof = 1))
print(np.std(x2, ddof = 1))

1.5811388300841898
15.811388300841896


In [6]:
# 변동 계수 비교 (자유도 = 0)

print(variation(x1)) 
print(variation(x2))

0.47140452079103173
0.4714045207910317


In [7]:
# 변동 계수 함수 직접 계산 (자유도 = 1)

print(np.std(x1, ddof = 1) / np.mean(x1))
print(np.std(x2, ddof = 1) / np.mean(x2))

0.5270462766947299
0.5270462766947299


## 스케일링

둘 이상의 변수의 값을 상대적으로 비교할 때 사용한다.  
상대적으로 비교하기 위해 각 데이터에 있는 값을 상대적인 값을 갖도록 변환한다.  

- Standard Scaling : x는 이론상 마이너스 무한대에서 무한대이다. 분포를 좀 더 고려해야 할 경우 사용한다.
    $$ \frac {x -\mu} {\sigma} $$
- Min-max Scaling(Zero - one scaling) : 항상 0-1사이의 값을 가진다. 최대 최소의 값까지 스케일이 중요한 경우 사용한다. (경험상 머신러닝에서 더 많이 보았다!!!)
    $$ \frac {x- \min(x)} {\max{x} - \min(x)}$$

스케일링은 변수간 비교 뿐만 아니라, 머신러닝에서도 널리 사용된다.  
sklean.preprocessing.은 머신러닝에서 더 쓰이기에 넘파이를 이용한 계산이 더 널리 쓰인다.  

## 파이썬을 이용한 스케일링

In [8]:
x1

array([1, 2, 3, 4, 5])

In [9]:
x2

array([10, 20, 30, 40, 50])

### 스탠다드 스케일링

```
(x - x.mean()) / x.std()  # x : ndarray
sklearn.preprocessing.StandardScaler
```

각 데이터가 0을 기준으로 어디에 위치하고 있는지를 한눈에 보기 쉽다. 

In [10]:
z1 = (x1 - x1.mean()) / x1.std()
z2 = (x2 - x2.mean()) / x2.std()

print(z1)
print(z2)

[-1.41421356 -0.70710678  0.          0.70710678  1.41421356]
[-1.41421356 -0.70710678  0.          0.70710678  1.41421356]


### 민맥스 스케일링
```
(x - x.mean()) / (x.max() - x.min())  # x : ndarray
sklearn.preprocessing.MinMaxScaler
```

In [11]:
z1 = (x1 - x1.min()) / (x1.max() - x1.min())
z2 = (x2 - x2.min()) / (x2.max() - x2.min())

print(z1)
print(z2)

[0.   0.25 0.5  0.75 1.  ]
[0.   0.25 0.5  0.75 1.  ]


## 사이킷런을 이용한 스케일링 
데이터프레임을 가지고 실습해본다. 

머신러닝에 이용하는데, 각 특징별로 스케일링이 되도록 최적화되어있다.  


In [12]:
# sklearn을 이용한 스케일링
X = pd.DataFrame({"X1":[1, 2, 3, 4, 5], 
                  "X2": [10, 20, 30, 40, 50]})

X

,X1,X2
0,1,10
1,2,20
2,3,30
3,4,40
4,5,50


In [13]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()     # 인스턴스화
Z = scaler.fit_transform(X) # fit_transform을 이용해 X를 변환 => 결과는 ndarray
pd.DataFrame(Z)             # 보기 좋게 데이터프레임화

,0,1
0,0.00,0.00
1,0.25,0.25
2,0.50,0.50
3,0.75,0.75
4,1.00,1.00


# 범위와 사분위 범위 계산 

범위와 사분위 범위는 산포를 나타내는 가장 직관적인 지표 중 하나이다.  
최대값과 최소값을 이용하는 범위는 이상치의 영향을 받을 수 있기 때문에 사분위 범위가 있다.  

- 범위 = 최대값 - 최소값
    - 직관적이지만 최대값과 최소값을 이용하는 범위는 이상치의 영향을 받을 수 있다.
- 사분위 범위 = 3사분위수 - 1사분위수  
    - IQR(interquartile range)라고도 하며, 이상치를 탐색할 때도 사용된다. 
    - 이상한 경우를 제외하고는 사분위 범위는 범위보다 작을 것.
    - **IQR Rule** : 변수별로 IQR 규칙을 만족하지 않는 샘플들을 판단하여 삭제하는 방법. 이상치 판단 구간은 다음과 같다.  
        - Q1 - IQR * 1.5 보다 작거나
        - Q3 + IQR * 1.5 보다 크거나


## 파이썬을 이용한 범위 및 사분위 범위 계산

평균이 100이고 표준편차가 20인 난수를 생성해 데이터를 만든다. 

In [14]:
x = np.random.normal(loc = 100, scale = 20, size = 1000)
x

array([125.95384376,  69.39248185, 104.71815439,  86.53152198,
        76.64116987,  72.28429864, 124.13912166,  99.72266635,
        91.34018791,  81.92211869, 111.371907  , 108.4316194 ,
       111.5217711 , 116.40887861,  99.7464101 ,  77.08789113,
       122.27377622, 105.62595   , 101.70779901, 131.63316427,
        93.40401928,  97.4784993 ,  96.56772312,  96.38124568,
       139.3153849 , 149.88218009,  94.185816  , 144.96766983,
       108.59949038, 105.4631064 , 132.51681492, 109.36226898,
        71.78389306, 131.00977855,  96.66799704,  68.98671519,
       122.03985849, 140.43280749, 110.68888564, 100.40191519,
       117.22105318, 128.84730032, 145.3715891 ,  80.9537168 ,
        71.34683345,  94.09903767,  91.61797178, 120.69278124,
        94.40584042, 105.78609124,  86.0223088 , 107.65171091,
       106.51426283,  95.85120557,  98.21851259, 117.99446577,
        93.21876964, 115.27848367,  90.61788448, 113.73034401,
        78.71673751,  82.56042479, 105.18551609, 105.11

### 범위
```
numpy.ptp(x)
numpy.max(X) - numpy.min(x) 
```

In [15]:
print(np.ptp(x))
print(np.max(x) - np.min(x))

150.12158384448978
150.12158384448978


### 사분위 범위
```
numpy.quantile(x, 0.75) - numpy.quantile(x, 0.25
scipy.stats.iqr(x)
```
범위의 경우 x의 값이 왔다갔다 하는 범위를 직관적으로 해석이 가능하나,  
사분위 범위는 25%~75% 사이가 이 정도 범위이라는 뜻이므로 덜 직관적인 단점이 있다. 

In [17]:
print(np.quantile(x, 0.75) - np.quantile(x, 0.25))
print(iqr(x))

27.182785510295744
27.182785510295744


# 별건 아니고


#### 생성된 난수의 IQR 밖의 값이 몇개나 있는지 구해볼까?
